In [1]:
import warnings
warnings.filterwarnings("ignore",category=FutureWarning)

import scanpy as sc
import numpy as np
from scipy import sparse
import random
import numpy as np
import scipy.sparse as sp
import pandas as pd
import anndata as ad

## Create new dataset

In [ ]:
data_path = "/cluster/work/bewi/data/tahoe100/h5ad/controls_Trametinib_mergedtahoe100_hvg.h5ad"
adata = sc.read_h5ad(data_path)

In [25]:
# PARAMETERS FOR THE SUBSET
export_path = "/cluster/work/bewi/members/rquiles/experiments/datasets/big_balanced_test.h5ad"

change_col_names = {"drug":"Agg_Treatment", "cell_line": "covariates"}

## UPDATE COLUMNS NEW DATASET
adata.obs = adata.obs.rename(columns=change_col_names)
adata.obs["control"] = adata.obs[change_col_names["drug"]] == control_name
adata.obs["control"] = adata.obs["control"].astype(int)
adata.uns["fields"] = []
adata.obs["dose"] = adata.obs["drugname_drugconc"].str.split(",").str[1].astype(float)

In [42]:
control_name = "DMSO_TF"
cell_lines_keep = np.unique(adata.obs["covariates"])[:3]
treatments_keep = list(np.unique(adata.obs["Agg_Treatment"])[[i for i in range(15)]])
if control_name not in treatments_keep:
    treatments_keep.append(control_name)

In [43]:
## SUBSET THE DATASET
idx = []
keep_rows = []

# First, find how many rows each cell_line–treatment pair has
for cell_line in cell_lines_keep:
    for treatment in treatments_keep:
        mask = (
            (adata.obs["covariates"] == cell_line)
            & (adata.obs["Agg_Treatment"] == treatment)
        )
        row_indexes = adata.obs[mask].index
        if len(row_indexes) > 0:
            keep_rows.extend(row_indexes)
            
# Randomize row order
random.shuffle(keep_rows)

# Subset the AnnData object
filtered_adata = adata[keep_rows, :]

In [44]:
## PREPROCESS AND EXPORT
sc.pp.normalize_total(filtered_adata, target_sum=1e4)
sc.pp.log1p(filtered_adata)
sc.pp.highly_variable_genes(filtered_adata, n_top_genes=2000, subset=True)
sc.pp.scale(filtered_adata, max_value=10)
filtered_adata = filtered_adata[:, filtered_adata.var.highly_variable]
filtered_adata.write(export_path)

/cluster/home/rquiles/miniconda3/envs/fcr-env/lib/python3.8/site-packages/scanpy/preprocessing/_normalization.py:169: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


## Inspect Observation Metadata

In [7]:
metadata_path = "/cluster/work/bewi/members/rquiles/experiments/datasets/obs_metadata.parquet"
df = pd.read_parquet(metadata_path)

In [11]:
df.columns

Index(['plate', 'BARCODE_SUB_LIB_ID', 'sample', 'gene_count', 'tscp_count',
       'mread_count', 'drugname_drugconc', 'drug', 'cell_line', 'sublibrary',
       'BARCODE', 'pcnt_mito', 'S_score', 'G2M_score', 'phase', 'pass_filter',
       'cell_name'],
      dtype='object')

In [43]:
df_3 = df[(df["cell_name"] == lines[0]) | (df["cell_name"] == lines[1]) | (df["cell_name"] == lines[2])]

In [61]:
drugs = np.unique(df_3["drug"])
drugs

array(['(R)-Verapamil (hydrochloride)', '(S)-Crizotinib',
       '18β-Glycyrrhetinic acid', '4EGI-1', '5-Azacytidine',
       '5-Fluorouracil', '8-Hydroxyquinoline', '9-ING-41', 'APTO-253',
       'AT7519', 'AZD-7648', 'AZD-8055', 'AZD1390', 'AZD2858',
       'Abemaciclib', 'Abiraterone acetate', 'Acetazolamide',
       'Acetohexamide', 'Adagrasib', 'Adenine', 'Adenosine', 'Afatinib',
       'Aliskiren', 'Allantoin', 'Allopurinol',
       'Almonertinib (hydrochloride)', 'Almonertinib (mesylate)',
       'Alpelisib', 'Altretamine', 'Amsacrine', 'Anastrozole',
       'Anethole trithione', 'Apalutamide', 'Aprepitant', 'Arbutin',
       'Artemether', 'Artesunate', 'Asciminib', 'Aspirin', 'Ataluren',
       'Atazanavir (sulfate)', 'Auranofin', 'Azithromycin (hydrate)',
       'BAY1125976', 'BI-3406', 'BI-78D3', 'Baicalin',
       'Balsalazide (sodium hydrate)', 'Belinostat', 'Belumosudil',
       'Belumosudil (mesylate)', 'Belzutifan', 'Bendamustine',
       'Benproperine (phosphate)', 'Ben

In [86]:
drugs_select = ["Trametinib", "Sapanisertib", "DMSO_TF"]
df_3_drugs = df_3[df_3["drug"].isin(drugs_select)]
df_3_drugs

,plate,BARCODE_SUB_LIB_ID,sample,gene_count,tscp_count,mread_count,drugname_drugconc,drug,cell_line,sublibrary,BARCODE,pcnt_mito,S_score,G2M_score,phase,pass_filter,cell_name
40956,plate10,69_001_023-lib_1681,smp_2427,1356,1883,2223,"[('Trametinib', 0.05, 'uM')]",Trametinib,CVCL_0023,lib_1681,69_001_023,0.048858,-0.229779,-0.194689,G1,full,A549
40973,plate10,69_007_095-lib_1681,smp_2427,1017,1412,1661,"[('Trametinib', 0.05, 'uM')]",Trametinib,CVCL_0023,lib_1681,69_007_095,0.085694,-0.354067,-0.333150,G1,full,A549
40998,plate10,69_019_163-lib_1681,smp_2427,1025,1374,1660,"[('Trametinib', 0.05, 'uM')]",Trametinib,CVCL_0028,lib_1681,69_019_163,0.050946,-0.244247,-0.213553,G1,full,AN3 CA
41045,plate10,69_035_042-lib_1681,smp_2427,911,1374,1617,"[('Trametinib', 0.05, 'uM')]",Trametinib,CVCL_0023,lib_1681,69_035_042,0.147016,-0.478583,-0.461538,G1,full,A549
41057,plate10,69_041_068-lib_1681,smp_2427,1734,2596,3051,"[('Trametinib', 0.05, 'uM')]",Trametinib,CVCL_0023,lib_1681,69_041_068,0.089753,-0.641262,-0.527289,G1,full,A549
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100648601,plate9,96_114_079-lib_2608,smp_2358,760,964,1140,"[('DMSO_TF', 0.0, 'uM')]",DMSO_TF,CVCL_0023,lib_2608,96_114_079,0.067427,-0.086124,-0.047253,G1,full,A549
100648612,plate9,96_117_142-lib_2608,smp_2358,875,1210,1391,"[('DMSO_TF', 0.0, 'uM')]",DMSO_TF,CVCL_0023,lib_2608,96_117_142,0.116529,-0.062315,0.010440,G2M,full,A549
100648620,plate9,96_121_124-lib_2608,smp_2358,960,1307,1514,"[('DMSO_TF', 0.0, 'uM')]",DMSO_TF,CVCL_0023,lib_2608,96_121_124,0.101760,-0.105377,-0.032418,G1,full,A549
100648740,plate9,96_175_074-lib_2608,smp_2358,765,1038,1203,"[('DMSO_TF', 0.0, 'uM')]",DMSO_TF,CVCL_0023,lib_2608,96_175_074,0.104046,-0.019253,-0.004396,G1,full,A549


In [70]:
df_3_drugs = df_3[(df_3["drug"] == "Trametinib") | (df_3["drug"] == "Sapanisertib") | (df_3["drug"] == "DMSO_TF")]
df_3_drugs["dose"] = df_3_drugs["drugname_drugconc"].str.split(",").str[1].astype(float)
df_3_drugs

/scratch/tmp.45352360.rquiles/ipykernel_4137876/667805940.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_3_drugs["dose"] = df_3_drugs["drugname_drugconc"].str.split(",").str[1].astype(float)


,plate,BARCODE_SUB_LIB_ID,sample,gene_count,tscp_count,mread_count,drugname_drugconc,drug,cell_line,sublibrary,BARCODE,pcnt_mito,S_score,G2M_score,phase,pass_filter,cell_name,dose
40956,plate10,69_001_023-lib_1681,smp_2427,1356,1883,2223,"[('Trametinib', 0.05, 'uM')]",Trametinib,CVCL_0023,lib_1681,69_001_023,0.048858,-0.229779,-0.194689,G1,full,A549,0.05
40973,plate10,69_007_095-lib_1681,smp_2427,1017,1412,1661,"[('Trametinib', 0.05, 'uM')]",Trametinib,CVCL_0023,lib_1681,69_007_095,0.085694,-0.354067,-0.333150,G1,full,A549,0.05
40998,plate10,69_019_163-lib_1681,smp_2427,1025,1374,1660,"[('Trametinib', 0.05, 'uM')]",Trametinib,CVCL_0028,lib_1681,69_019_163,0.050946,-0.244247,-0.213553,G1,full,AN3 CA,0.05
41045,plate10,69_035_042-lib_1681,smp_2427,911,1374,1617,"[('Trametinib', 0.05, 'uM')]",Trametinib,CVCL_0023,lib_1681,69_035_042,0.147016,-0.478583,-0.461538,G1,full,A549,0.05
41057,plate10,69_041_068-lib_1681,smp_2427,1734,2596,3051,"[('Trametinib', 0.05, 'uM')]",Trametinib,CVCL_0023,lib_1681,69_041_068,0.089753,-0.641262,-0.527289,G1,full,A549,0.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100648601,plate9,96_114_079-lib_2608,smp_2358,760,964,1140,"[('DMSO_TF', 0.0, 'uM')]",DMSO_TF,CVCL_0023,lib_2608,96_114_079,0.067427,-0.086124,-0.047253,G1,full,A549,0.00
100648612,plate9,96_117_142-lib_2608,smp_2358,875,1210,1391,"[('DMSO_TF', 0.0, 'uM')]",DMSO_TF,CVCL_0023,lib_2608,96_117_142,0.116529,-0.062315,0.010440,G2M,full,A549,0.00
100648620,plate9,96_121_124-lib_2608,smp_2358,960,1307,1514,"[('DMSO_TF', 0.0, 'uM')]",DMSO_TF,CVCL_0023,lib_2608,96_121_124,0.101760,-0.105377,-0.032418,G1,full,A549,0.00
100648740,plate9,96_175_074-lib_2608,smp_2358,765,1038,1203,"[('DMSO_TF', 0.0, 'uM')]",DMSO_TF,CVCL_0023,lib_2608,96_175_074,0.104046,-0.019253,-0.004396,G1,full,A549,0.00


In [71]:
# Check which plates need to be used
np.unique(df_3_drugs["plate"])

array(['plate1', 'plate10', 'plate11', 'plate12', 'plate13', 'plate14',
       'plate2', 'plate3', 'plate4', 'plate5', 'plate6', 'plate7',
       'plate8', 'plate9'], dtype=object)

In [73]:
# Read the file in backed mode
plate = "/cluster/work/bewi/data/tahoe100/h5ad/plate1_filt_Vevo_Tahoe100M_WServicesFrom_ParseGigalab.h5ad"
adata = sc.read_h5ad(plate, backed="r")

In [94]:
ids_plate1 = df_3_drugs[df_3_drugs["plate"] == "plate1"]["BARCODE_SUB_LIB_ID"].values
subset = adata[adata.obs_names.isin(ids_plate1), :].to_memory()

In [88]:
ad.concat([subset], join="outer")

AnnData object with n_obs × n_vars = 14141 × 62710
    obs: 'sample', 'gene_count', 'tscp_count', 'mread_count', 'drugname_drugconc', 'drug', 'cell_line', 'sublibrary', 'BARCODE', 'pcnt_mito', 'S_score', 'G2M_score', 'phase', 'pass_filter', 'cell_name', 'plate'

In [92]:
subset.obs.head()

,sample,gene_count,tscp_count,mread_count,drugname_drugconc,drug,cell_line,sublibrary,BARCODE,pcnt_mito,S_score,G2M_score,phase,pass_filter,cell_name,plate
BARCODE_SUB_LIB_ID,,,,,,,,,,,,,,,,
35_010_059-lib_841,smp_1529,2085,3618,4309,"[('Trametinib', 0.05, 'uM')]",Trametinib,CVCL_0023,lib_841,35_010_059,0.112769,-0.023810,1.651099,G2M,full,A549,plate1
35_021_151-lib_841,smp_1529,576,811,959,"[('Trametinib', 0.05, 'uM')]",Trametinib,CVCL_0023,lib_841,35_021_151,0.083847,-0.019048,-0.042857,G1,full,A549,plate1
35_024_112-lib_841,smp_1529,1661,2815,3342,"[('Trametinib', 0.05, 'uM')]",Trametinib,CVCL_0023,lib_841,35_024_112,0.148845,-0.028571,0.367033,G2M,full,A549,plate1
35_027_099-lib_841,smp_1529,1069,1489,1729,"[('Trametinib', 0.05, 'uM')]",Trametinib,CVCL_0069,lib_841,35_027_099,0.106111,-0.071429,-0.032967,G1,full,SK-MEL-2,plate1
35_033_165-lib_841,smp_1529,1959,3349,3968,"[('Trametinib', 0.05, 'uM')]",Trametinib,CVCL_0023,lib_841,35_033_165,0.105106,-0.104762,0.588462,G2M,full,A549,plate1


In [96]:
n = subset.n_obs  # number of cells

# Generate a random permutation of indices
perm = np.random.permutation(n)

# Apply the permutation to reorder rows (cells)
subset = subset[perm, :].copy()
subset.obs.head()

,sample,gene_count,tscp_count,mread_count,drugname_drugconc,drug,cell_line,sublibrary,BARCODE,pcnt_mito,S_score,G2M_score,phase,pass_filter,cell_name,plate
BARCODE_SUB_LIB_ID,,,,,,,,,,,,,,,,
96_175_025-lib_885,smp_1590,759,986,1176,"[('DMSO_TF', 0.0, 'uM')]",DMSO_TF,CVCL_0023,lib_885,96_175_025,0.051724,-0.004762,-0.014423,G1,full,A549,plate1
95_056_192-lib_879,smp_1589,1846,3106,3648,"[('DMSO_TF', 0.0, 'uM')]",DMSO_TF,CVCL_0023,lib_879,95_056_192,0.088860,-0.163690,1.583516,G2M,full,A549,plate1
96_002_017-lib_855,smp_1590,987,1225,1462,"[('DMSO_TF', 0.0, 'uM')]",DMSO_TF,CVCL_0028,lib_855,96_002_017,0.037551,-0.001035,-0.033309,G1,full,AN3 CA,plate1
96_060_030-lib_861,smp_1590,919,1221,1492,"[('DMSO_TF', 0.0, 'uM')]",DMSO_TF,CVCL_0023,lib_861,96_060_030,0.082719,-0.010073,-0.057416,G1,full,A549,plate1
43_007_013-lib_880,smp_1537,1165,1616,1948,"[('Sapanisertib', 0.05, 'uM')]",Sapanisertib,CVCL_0069,lib_880,43_007_013,0.035272,0.071087,0.928230,G2M,full,SK-MEL-2,plate1


In [101]:
len(subset.obs[subset.obs["drug"] == "DMSO_TF"])

8302

In [102]:
len(subset.obs)

14141

In [118]:
lines_select = np.unique(subset.obs["cell_name"])
drugs_select = np.unique(subset.obs["drug"])
print(lines_select)
print(drugs_select)

['A549' 'AN3 CA' 'SK-MEL-2']
['DMSO_TF' 'Sapanisertib' 'Trametinib']


In [113]:
## SUBSET THE DATASET
adata = subset
idx = []
keep_rows = []

lines_select = np.unique(subset.obs["cell_name"])
control_name = "DMSO_TF"

# First, find how many rows each cell_line–treatment pair has
for cell_line in lines_select:
    mask = (
        (adata.obs["cell_name"] == cell_line)
        & (adata.obs["drug"] != control_name)
    )
    row_indexes = adata.obs[mask].index

    # Keep all treated samples
    n_treated = len(row_indexes)
    if len(row_indexes) > 0:
        keep_rows.extend(row_indexes)

    # Keep number of control samples = 1/3 treated samples
    n_controls = n_treated//3
    mask = (
        (adata.obs["cell_name"] == cell_line)
        & (adata.obs["drug"] == control_name)
    )
    row_indexes = adata.obs[mask].index
    row_indexes = row_indexes[:n_controls]
    if len(row_indexes) > 0:
        keep_rows.extend(row_indexes)

# Randomize row order
random.shuffle(keep_rows)

# Subset the AnnData object
filtered_adata = adata[keep_rows, :]

In [116]:
len(filtered_adata.obs[filtered_adata.obs["drug"] == "DMSO_TF"])

1946

In [117]:
len(filtered_adata.obs[filtered_adata.obs["drug"] != "DMSO_TF"])

5839

## Testing Area

In [4]:
adata1 = sc.read_h5ad("/cluster/work/bewi/members/rquiles/experiments/datasets/3_cells_2_drugs.h5ad")
adata2 = sc.read_h5ad("/cluster/work/bewi/members/rquiles/experiments/datasets/3_cells_2_drugs_balanced.h5ad")
adata3 = sc.read_h5ad("/cluster/work/bewi/members/rquiles/experiments/datasets/test_3cell_lines_balanced.h5ad")


print(f"Number samples adata1: {len(adata1.obs)}")
print(f"Number samples adata2: {len(adata2.obs)}")
print(f"Number samples adata3: {len(adata3.obs)}")

Number samples adata1: 43301
Number samples adata2: 20488
Number samples adata3: 4725


In [2]:
adata = sc.read_h5ad("/cluster/work/bewi/members/rquiles/experiments/datasets/3_cells_2_drugs_balanced.h5ad")

In [4]:
adata.obs

,sample,gene_count,tscp_count,mread_count,drugname_drugconc,Agg_Treatment,cell_line,sublibrary,BARCODE,pcnt_mito,S_score,G2M_score,phase,pass_filter,covariates,plate,control,dose
BARCODE_SUB_LIB_ID,,,,,,,,,,,,,,,,,,
43_035_074-lib_852,smp_1537,3293,6292,7699,"[('Sapanisertib', 0.05, 'uM')]",Sapanisertib,CVCL_0069,lib_852,43_035_074,0.057057,-0.339827,-0.267483,G1,full,SK-MEL-2,plate1,0,0.05
35_132_106-lib_852,smp_1529,1307,1767,2148,"[('Trametinib', 0.05, 'uM')]",Trametinib,CVCL_0028,lib_852,35_132_106,0.046972,0.185122,0.043890,S,full,AN3 CA,plate1,0,0.05
35_144_073-lib_899,smp_1529,860,1065,1292,"[('Trametinib', 0.05, 'uM')]",Trametinib,CVCL_0028,lib_899,35_144_073,0.072300,0.080656,-0.004509,S,full,AN3 CA,plate1,0,0.05
96_002_171-lib_866,smp_1590,2044,3180,3923,"[('DMSO_TF', 0.0, 'uM')]",DMSO_TF,CVCL_0069,lib_866,96_002_171,0.055975,-0.068452,0.833516,G2M,full,SK-MEL-2,plate1,1,0.00
35_043_134-lib_850,smp_1529,1439,2270,2755,"[('Trametinib', 0.05, 'uM')]",Trametinib,CVCL_0023,lib_850,35_043_134,0.114978,-0.128571,-0.119341,G1,full,A549,plate1,0,0.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43_118_064-lib_880,smp_1537,1359,1790,2152,"[('Sapanisertib', 0.05, 'uM')]",Sapanisertib,CVCL_0028,lib_880,43_118_064,0.036872,-0.034062,0.154582,G2M,full,AN3 CA,plate1,0,0.05
43_184_100-lib_897,smp_1537,1359,2017,2440,"[('Sapanisertib', 0.05, 'uM')]",Sapanisertib,CVCL_0069,lib_897,43_184_100,0.067923,-0.080952,-0.109341,G1,full,SK-MEL-2,plate1,0,0.05
96_106_157-lib_868,smp_1590,4275,8871,10769,"[('DMSO_TF', 0.0, 'uM')]",DMSO_TF,CVCL_0069,lib_868,96_106_157,0.068312,-0.190476,1.493559,G2M,full,SK-MEL-2,plate1,1,0.00


In [5]:
print(f"A549: {len(adata.obs[adata.obs['cell_name'] == 'A549'])}")
print(f"SK-MEL-2: {len(adata.obs[adata.obs['cell_name'] == 'SK-MEL-2'])}")
print(f"AN3 CA: {len(adata.obs[adata.obs['cell_name'] == 'AN3 CA'])}")

A549: 7683
SK-MEL-2: 7683
AN3 CA: 5122


In [6]:
print(f"Trametinib: {len(adata.obs[adata.obs['Agg_Treatment'] == 'Trametinib'])}")
print(f"Sapanisertib: {len(adata.obs[adata.obs['Agg_Treatment'] == 'Sapanisertib'])}")
print(f"DMSO_TF: {len(adata.obs[adata.obs['Agg_Treatment'] == 'DMSO_TF'])}")

Trametinib: 9792
Sapanisertib: 5543
DMSO_TF: 5153


In [6]:
for drug in ["Trametinib", "Sapanisertib"]:
    for dose in [0.5, 5]:
        try:
            print(f"{drug} ({dose}): {len(adata.obs[(adata.obs['Agg_Treatment'] == drug) & (adata.obs['dose'] == dose)])}")
        except Exception as e:
            print(f"Could not print {drug} ({dose})")

Trametinib (0.5): 2742
Trametinib (5): 4794
Sapanisertib (0.5): 1689
Sapanisertib (5): 2085


In [9]:
cell_lines = np.unique(adata.obs["cell_name"].values)
drugs = np.unique(adata.obs["Agg_Treatment"].values)

for line in cell_lines:
    for drug in drugs:
        n_instances = len(adata.obs[(adata.obs['Agg_Treatment'] == drug) & (adata.obs['cell_name'] == line)])
        print(f"{line} - {drug}: {n_instances}")

A549 - DMSO_TF: 1922
A549 - Sapanisertib: 2034
A549 - Trametinib: 3727
AN3 CA - DMSO_TF: 1280
AN3 CA - Sapanisertib: 1234
AN3 CA - Trametinib: 2608
SK-MEL-2 - DMSO_TF: 1951
SK-MEL-2 - Sapanisertib: 2275
SK-MEL-2 - Trametinib: 3457


In [5]:
adata.X[100:200, 100:200].toarray()

array([[0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 2.6131697, 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       ...,
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ]], dtype=float32)

In [36]:
print("Nonzero entries:", np.count_nonzero(adata.X))
print("Fraction nonzero:",  np.count_nonzero(adata.X) / adata.X.size)

Nonzero entries: 40976000
Fraction nonzero: 1.0


In [127]:
np.unique(adata.obs[adata.obs["Agg_Treatment"] == "Trametinib"]["dose"])

array([0.05, 0.5 , 5.  ])

In [7]:
np.unique(adata.obs[adata.obs["Agg_Treatment"] == "DMSO_TF"]["dose"])

array([0.])

### piSCVI Dataset

In [3]:
adata_p = sc.read_h5ad("/cluster/work/bewi/members/rquiles/piscvi/data/NBsmall/NB.bone.Met_preprocessed.h5ad")

In [23]:
X = adata_p.layers["RNA"].toarray()
X_log = adata_p.layers["log1p_norm"].toarray()

In [32]:
layer = adata_p.layers["RNA"]
if sp.issparse(layer):
    print("Nonzero entries:", layer.nnz)
    print("Fraction nonzero:", layer.nnz / (layer.shape[0] * layer.shape[1]))

Nonzero entries: 59982036
Fraction nonzero: 0.06490262479974289


In [33]:
layer = adata_p.layers["log1p_norm"]
if sp.issparse(layer):
    print("Nonzero entries:", layer.nnz)
    print("Fraction nonzero:", layer.nnz / (layer.shape[0] * layer.shape[1]))

Nonzero entries: 59982036
Fraction nonzero: 0.06490262479974289


### Counts dataset

In [38]:
adata_c = sc.read_h5ad("/cluster/work/bewi/members/rquiles/experiments/datasets/3_cells_2_drugs_balanced_counts.h5ad")

<20488x5000 sparse matrix of type '<class 'numpy.float32'>'
	with 5389649 stored elements in Compressed Sparse Row format>

## LEGACY CODE

In [22]:
## SUBSET THE DATASET
idx = []
keep_rows = []

pair_counts = []

# First, find how many rows each cell_line–treatment pair has
for cell_line in cell_lines_keep:
    for treatment in treatments_keep:
        mask = (
            (adata.obs["covariates"] == cell_line)
            & (adata.obs["drugname_drugconc"] == treatment)
        )
        row_indexes = adata.obs[mask].index
        if len(row_indexes) > 0:
            pair_counts.append(len(row_indexes))

# Find the smallest group size
min_size = min(pair_counts)

# Sample each group in a balanced way
for cell_line in cell_lines_keep:
    for treatment in treatments_keep:
        mask = (
            (adata.obs["covariates"] == cell_line)
            & (adata.obs["drugname_drugconc"] == treatment)
        )
        row_indexes = np.array(adata.obs[mask].index).tolist()
        if len(row_indexes) >= min_size:
            sampled_rows = random.sample(row_indexes, min_size)
            keep_rows.extend(sampled_rows)

# Randomize row order
random.shuffle(keep_rows)

# Subset the AnnData object
filtered_adata = adata[keep_rows, :]